# Feature extraction and reverse image search using pre-trained Deep Convolutional Neural Networks

This notebook deals with the procedure of analyzing a large set of images using a pre-trained convolutional network, extracting feature vectors (activations of last layer) for each one which represent each image. 

After the analysis is done, we will review some retrieval tasks that you can do with such an analysis. The main task will be that of "reverse image search," which refers to searching for the most similar set of images to some query image. 


### Step 0: Make a GPU-enabled environment (via the terminal, before runnning python code)

In [1]:
!module list
################################WITH CONDA

#1 step create environment
# conda -n gputest python=3

#2 activate environment
#conda activate gputest

#3 install kernel for the gpu enbaling (?!)
# pip install ipykernel

#4 set kernel
# python -m ipykernel install --user --name gputest --display-name "gputest"

#5 get tensorflow for gpu running
#conda install tensorflow-gpu

#6 install jupyter environment
#conda install jupyter

# install keras library
#pip install keras 

'''It's important to do pip install keras and \
not conda install keras because otherwise it would downgrade the \
tensorflow module.'''
# 7 run the notebook
#jupyter notebook


#################################### WITH virtualenv


#1  Build environment
#virtualenv --system-site-packages targetDirectory 
#2 Activate environment
# source ~/targetDirectory/bin/activate
#


Currently Loaded Modules:
  1) astro   2) cuda/11.2   3) python/anaconda3/2021.05

 



"It's important to do pip install keras and not conda install keras because otherwise it would downgrade the tensorflow module."


### Step 1: Import necessary modules and write main functions


In [2]:
import os
import tensorflow
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import decode_predictions, preprocess_input
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
from keras.utils.vis_utils import plot_model
from IPython.display import Image 
import time
import random
from sklearn.decomposition import PCA
from scipy.spatial import distance
from sklearn.neighbors import NearestNeighbors
Image.MAX_IMAGE_PIXELS = None


# config = tensorflow.ConfigProto()
# config.gpu_options.allow_growth = True
# sess = tensorflow.Session(config=config)



def conv_model(model_name='resnet',show_model=True):
    if model_name=='inception_v3': #works
        from keras.applications.inception_v3 import InceptionV3
        model = tensorflow.keras.applications.InceptionV3(weights='imagenet', include_top=True)
    elif model_name=='vgg16':   #works 
        from tensorflow.keras.applications.vgg16 import VGG16
        model = tensorflow.keras.applications.VGG16(weights='imagenet', include_top=True)
    elif model_name=='xception': #works
        from keras.applications.xception import Xception
        model = tensorflow.keras.applications.Xception(weights='imagenet', include_top=True)
    elif model_name=='resnet': #doesn't work
        from keras.applications.resnet50 import ResNet50
        model = tensorflow.keras.applications.ResNet50(weights='imagenet', include_top=True)
    elif model_name=='vgg': #works
        model = tensorflow.keras.applications.VGG19(weights='imagenet', include_top=True)
    elif model_name=='MobileNet': #doesn't work
        from keras.applications.mobilenet import MobileNet
        model = tensorflow.keras.applications.MobileNet(weights='imagenet', include_top=True)
    else:
        print('Incorrect model_name fed to function conv_model')
    if model!=None:
        for layer in model.layers:
            layer.trainable=False
    if show_model==True:
        model.summary()
    return model


def delete_model(model, clear_session=True):
    '''removes model!
    '''
    del model
    gc.collect()
    if clear_session: K.clear_session()

def get_neighbors(feature_vector,feature_vectors, k=5):
    '''Function that returns the neighbor distances and their indeces (of the image array corresponding to the images names)'''
    similar_idx = [ distance.euclidean(feature_vector, feat) for feat in feature_vectors ]
    idx_closest = sorted(range(len(similar_idx)), key=lambda k: similar_idx[k])[1:1+k]
    distances=[]
    for i in idx_closest:
        distances.append(similar_idx[i])
    distances=np.array(distances)
    idx_closest=np.array(idx_closest)
    return distances,idx_closest        


def load_image_v2(path,model_name='resnet'):
    '''An effort to write a different function to load the model, since I get a bug when trying to load more than 500 pictures.'''
    img = tf.keras.preprocessing.image.load_img(path, grayscale=False, color_mode='rgb', target_size=model.input_shape[1:3],interpolation='nearest')
    input_arr = tf.keras.preprocessing.image.img_to_array(img)
    x = np.array([input_arr])  # Convert single image to a batch
    x = np.expand_dims(x, axis=0)
    if model=='inception_v3':
        from keras.applications.inception_v3 import preprocess_input as process_inception
        x=preprocess_inception(x)
    elif model_name=='vgg16':
        from keras.applications.vgg16 import preprocess_input as process_vgg16
        print('shape of vgg16 input:',np.shape(x))
        x=process_vgg16(x)
    elif model_name=='xception':
        from keras.applications.xception import preprocess_input as process_xception
        x=process_xception(x)
    elif model_name=='resnet':
        from keras_applications.resnet import preprocess_input as process_resnet
        print('shape of resnet input:',np.shape(x))
        x=process_resnet(x)
    elif model_name=='vgg':
        from keras.applications.vgg19 import preprocess_input as process_vgg
        x=process_vgg(x)
    elif model_name=='mobile':
        from keras.applications.mobilenet import preprocess_input as process_mobile
        x=process_mobile(x)  
    else:
        print('Incorrect  model_name fed to function load_image')
    return img, x


def load_image(path,model_name='resnet'):
    '''Load an image an preprocess it, and return the image and the preprocessed input'''
    img = image.load_img(path, target_size=model.input_shape[1:3])
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    if model=='inception_v3':
        from keras.applications.inception_v3 import preprocess_input as process_inception
        x=preprocess_inception(x)
    elif model_name=='vgg16':
        from keras.applications.vgg16 import preprocess_input as process_vgg16
        print('shape of vgg16 input:',np.shape(x))
        x=process_vgg16(x)
    elif model_name=='xception':
        from keras.applications.xception import preprocess_input as process_xception
        x=process_xception(x)
    elif model_name=='resnet':
        from keras_applications.resnet import preprocess_input as process_resnet
        print('shape of resnet input:',np.shape(x))
        x=process_resnet(x)
    elif model_name=='vgg':
        from keras.applications.vgg19 import preprocess_input as process_vgg
        x=process_vgg(x)
    elif model_name=='mobile':
        from keras.applications.mobilenet import preprocess_input as process_mobile
        x=process_mobile(x)  
    else:
        print('Incorrect  model_name fed to function load_image')
    return img, x

def img_to_conv_features(model_name,model,x):
    '''Convert 1 image to its convolutional features'''
    if model_name=='inception_v3':
        feat_extractor = Model(inputs=model.input, outputs=model.get_layer("avg_pool").output)
    elif model_name=='vgg16':
        feat_extractor = Model(inputs=model.input, outputs=model.get_layer("fc2").output)
        feat = feat_extractor.predict(x)
    elif model_name=='xception':
        feat_extractor = Model(inputs=model.input, outputs=model.get_layer("avg_pool").output)
    elif model_name=='resnet':
        feat_extractor = Model(inputs=model.input, outputs=model.get_layer("fc2").output)
        feat = feat_extractor.predict(x)
    elif model_name=='vgg':
        feat_extractor = Model(inputs=model.input, outputs=model.get_layer("fc2").output)
    else:
        print('Incorrect  model_name fed to function img_to_conv_features')
    feat=feat_extractor(x)
    return feat
def imgs_to_conv_features(model_name,model,images,image_path):
    '''Convert a batch of images into its convolutional features'''
    if model_name=='inception_v3':
        feat_extractor = Model(inputs=model.input, outputs=model.get_layer("avg_pool").output)
    elif model_name=='vgg16':
        feat_extractor = Model(inputs=model.input, outputs=model.get_layer("fc2").output)
    elif model_name=='xception':
        feat_extractor = Model(inputs=model.input, outputs=model.get_layer("avg_pool").output)
    elif model_name=='resnet':
        feat_extractor = Model(inputs=model.input, outputs=model.get_layer("fc2").output)
    elif model_name=='vgg':
        feat_extractor = Model(inputs=model.input, outputs=model.get_layer("fc2").output)
    elif model_name=='MobileNet':
        feat_extractor = Model(inputs=model.input, outputs=model.get_layer("fc2").output)
    else:
        print('Incorrect  model_name fed to function img_to_conv_features')
    tic = time.process_time()

    every=20
    features = []
    for i, image_path in enumerate(images):

        if i % every == 0:
            toc = time.process_time()
            elap = toc-tic;
            remaining_time=(len(images)-i)*elap/every
            hours=remaining_time//3600
            minutes=remaining_time//60
            seconds=remaining_time%60
            print("analyzing image %d / %d. Time/%d pics: %4.4f seconds." % (i, len(images),every,elap))
            print('Remaining time: %d hours %d minutes %d sec.' %(hours,minutes,seconds))
            
            tic = time.process_time()
        img, x = load_image(path=image_path,model_name=model_name);
        #print(image_path+"\n")
        feat = feat_extractor.predict(x,batch_size=32)[0]
        features.append(feat)

    print('finished extracting features for %d images' % len(images))
    return features


def choose_imgs(max_num_images=2000,images_path='C:\\Users\\Rami\\Desktop\\PetImages\\dogs-vs-cats'):
    image_extensions = ['.jpg', '.png', '.jpeg']   # case-insensitive (upper/lower doesn't matter)
    images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(images_path) for f in filenames if os.path.splitext(f)[1].lower() in image_extensions]
    print(np.shape(images))
    if max_num_images < len(images):
        images = [images[i] for i in sorted(random.sample(range(len(images)), max_num_images))]

    print("keeping %d images to analyze" % len(images))
    return images
def reduce_PCA(features,n_components=40):
    features = np.array(features)
    pca = PCA(n_components=n_components)
    pca.fit(features)
    pca_features = pca.transform(features)
    return pca_features

def plot_activation_layer(feature_vector):
    
    print(np.shape(feature_vector))
    feature_vector=np.array(feature_vector)
    feature_vector=feature_vector.flatten()
    plt.figure(figsize=(16,4))
    plt.ylabel('Activations of last layer (fc2)')
    plt.xlabel('## of neuron')
    plt.plot(feature_vector)
    plt.show()
    plt.close()
    
    return 0

def get_concatenated_images(indexes, thumb_height):
    thumbs = []
    for idx in indexes:
        img = image.load_img(images[idx])
        img = img.resize((int(img.width * thumb_height / img.height), thumb_height))
        thumbs.append(img)
    concat_image = np.concatenate([np.asarray(t) for t in thumbs], axis=1)
    return concat_image

2021-10-03 22:46:56.472585: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


### Step 2: Check version of tensorflow and Default running device (GPU OR CPU)

In [3]:
# !pwd
# from distutils.version import LooseVersion
# import warnings

# assert LooseVersion(tensorflow.__version__)>=LooseVersion('1.0'), 'Please use Tensorflow'
# print('TensorFlow Version: {}'.format(tensorflow.__version__))

# #Check for gpu
# if not tensorflow.test.gpu_device_name():
#     warnings.warn('No GPU found. Please ensure you have installed  TensorFlow correctly')
# else: 
#     print('Default GPU Device: {}'.format(tensorflow.test.gpu_device_name()))
    
# device_name=tensorflow.test.gpu_device_name()
# gpu_devices = tensorflow.config.list_physical_devices('GPU')
# print("Num GPUs:", len(gpu_devices))
# cpu_devices = tensorflow.config.list_physical_devices('CPU')
# print("Num CPUs:", len(cpu_devices))

# from tensorflow.python.client import device_lib 
# print(device_lib.list_local_devices())


## Test if tensorflow is built with gpu support
# 
# print(tensorflow.test.is_built_with_gpu_support())

#Get number of threads used for parallelism between independent operations.
#tf.config.threading.get_inter_op_parallelism_threads()

#Get number of threads used within an individual op for parallelism.
#tf.config.threading.get_intra_op_parallelism_threads()

#Determines the number of threads used by independent non-blocking operations. 0 means the system picks an appropriate number.
#tf.config.threading.set_inter_op_parallelism_threads(    num_threads)


#Set number of threads used within an individual op for parallelism.
#Certain operations like matrix multiplication and reductions can utilize parallel threads for speed ups. A value of 0 means the system picks an appropriate number.

#tf.config.threading.set_intra_op_parallelism_threads(    num_threads)




In [4]:

# def sizeof_fmt(num, suffix='B'):
#     for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
#         if abs(num) < 1024.0:
#             return "%3.1f %s%s" % (num, unit, suffix)
#         num /= 1024.0
#     return "%.1f%s%s" % (num, 'Yi', suffix)

# for d in devices:
#     t = d.device_type
#     name = d.physical_device_desc
#     l = [item.split(':',1) for item in name.split(", ")]
#     name_attr = dict([x for x in l if len(x)==2])
#     dev = name_attr.get('name', 'Unnamed device')
#     print(f" {d.name} || {dev} || {t} || {sizeof_fmt(d.memory_limit)}")

# BATCH_SIZE = 32
# GPUS = ["GPU:0"]

# def process(image, label):
#     image = tf.image.resize(image, [299, 299]) / 255.0
#     return image, label

# strategy = tf.distribute.MirroredStrategy( GPUS )
# print('Number of devices: %d' % strategy.num_replicas_in_sync) 

# batch_size = BATCH_SIZE * strategy.num_replicas_in_sync

# dataset = dataset.map(process).shuffle(500).batch(batch_size)



# tf.get_logger().setLevel('ERROR')

# start = time.time()
# with strategy.scope():

Here we start to set which image we want to compare to which folder of images. We can also choose the model and get some useful messages

### A little bit about the context

The networks that are implemented are some of the top performing on the classification contest by imagenet on classifying  
pictures on 1000 labels , when fed with a dataset 1.2 milion pictures. 
Since the machine learning consists of 


Step 1: Designing the architechture of your network


(how many layers, how many neurons (units) does a layer have, which
connections exist between adjacent layers, and what is the mathematical operation that a layer operates on the input in order
to produce the output (activation function) which wiill be fed to the next layer.

Step 2: Train the model

  Start by initializing some numbers for all weight (for example set them all to 0), and then try to find optimal values
for these parameters, the weights between the connections. The basic idea on how to find these optimal values,
is to define a loss function which represents  how well the programm is achieving what we want it to do. We want the loss
function to be small when the program is accomplishing the task (classifying images with labels correctly)
and we want the loss to be large when the program is failing (you give it a picture of a cat, and it tells you its a dog).
Since the loss function of the CNN is dependant by some mathematic expression to the weights of the network, minimizing is
an optimization problem which can be solved by just walking towards the negative derivative of L(W1,w2..,W_n).
A common algorithm that does this downhill walk on ths multidimensional space of weights in order
to find the optimal values of the weights is stohastic gradient descent (SGD)


Step 3: Your model is trained now and has small loss. 

Now the cnn can receive new input images and be able to produce "signature-information" containing feature vectors.


Propagate the input x in the network and return the second-last layer, commonly called feature vector (denamed by 
  variable feature here). The feature vector has ~10^3 dimensions and simply consists of all the weight between the last 2 
layers.  It is common in convolutional neural network (CNN) architecture that the two last layers are fully
conected, while the rest of the layers inside the convolutional network (convolutions,max pool, avg pool, etc.) can skip
connections from a layer to the next one.


Remember that the networks that we are implementing have very high accuracy on classifying 1000 different types of objects
which means that the layer before the last inputs some vector and outputs it's activation function to the last layer,
which is the 1000 element layer of the predictions. Because of this, the feature vector is a better representation of the 
image's content than simply the 2D X3 (rgb) matrix of the colors consisting the image in order to assess a similarity measure.


We dont use the last layer of the network (predictions) but the second last (feature vector)

Another way to find the closest pictures could also be to say: let's find the pictures in the dataset which are the closest to our own by distance on their vector space. These ones with the least distance will be classified as best matches for the reverse image search. We will use the 5-nearest-neighbours algorithm, which in general for k neighbours is called kNN (k nearest neighbours).

### K-nearest neighbors algorithm


In [5]:
!pwd

/lustre/hpc/astro/rami/tensorflow_gpu


### Step 3: Run the neural network
### All  the code in one cell without many outputs

In [ ]:
# Parameters

##########################################################
#Set the path to the image, images and max number of imgs#
path='/groups/astro/rami/images/image_000005.png'        #
image_path='/groups/astro/rami/images/'                  #
max_num_images=20                                      #
#Which Deep Neural network to use                        #
model_name='vgg'                                         #
#How to compare the features(fingerprints)               #
distance_metric='knn'                                    #
k=5                                                      #
#Reduce dimensionality of data?                          #
use_PCA=False                                            # 
n_components=500                                         #
                                                         #
#True for calculating feature vectors                    #   
#for the entire dataset                                  #
recalculate_dataset=True                                 #
#Show the output on jupyter?                             #
display_output=True                                      #
#Save the output somewhere?                              #
save_output=False                                        #
output_folder='/lustre/hpc/astro/rami/images/'           #
                                                         #
use_timing=True                                          #
                                                         #
                                                         #
                                                         #
                                                         #
###########################################################




print("Running search with image:%s \n comparing to %d images \n which are in folder %s, \n using model %s \n and distance metric:\
%s with %d nearest neighbors. \n PCA used?:%s \n recalculating dataset?%s "%(path,max_num_images,image_path,model_name,distance_metric,k,use_PCA,recalculate_dataset))





### Initializing model
if use_timing:
    t0 = time.time()
model=conv_model(model_name=model_name)  






# Calculating target image (observation) N.N.-fingerprint
if use_timing:
    t1 = time.time()
img, x = load_image(path=path,model_name=model_name)
if use_timing:
    t11=time.time()
x=np.array(x)
if use_timing:
    t12=time.time()
feature=img_to_conv_features(model_name=model_name,model=model,x=x)
if use_timing:
    t13=time.time()
feature=np.array(feature)
if use_timing:
    t14=time.time()
print("Shape of feature:",np.shape(feature))







# Caclulating dataset images (simulations) N.N.-fingerprint
if use_timing:
    t2 = time.time()
if recalculate_dataset==True:
    images=choose_imgs(max_num_images=max_num_images,images_path=image_path)
if use_timing:
    t3 = time.time()
if recalculate_dataset==True:
    features=imgs_to_conv_features(model_name=model_name,model=model,images=images,image_path=image_path)
print("Shape of features:",np.shape(features))
print('\n')


# Possibly reduce dimensionality of data
if use_timing:
    t4 = time.time()
if use_PCA==True:
    full_features=np.vstack((feature,features))
    reduced_full_features=reduce_PCA(full_features,n_components=n_components)
    
    reduced_feature=reduced_full_features[0,:]
    reduced_features=reduced_full_features[1:,:]
elif use_PCA==False:
    reduced_features=features
    reduced_feature=feature
if use_timing:
    t5 = time.time()    
reduced_feature=np.array(reduced_feature)






# Calculate distances between target-dataset, and plot the best 5 matches
if distance_metric=='knn':
    distances,idx_closest=get_neighbors(feature_vector=reduced_feature,feature_vectors=reduced_features,k=k)
    distances=distances.flatten()
    idx_closest=idx_closest.flatten()
    distances=np.array(distances)
    target_image=path[-16:]
    
    print('\n')
    print('neighbour indeces with closest distances with respect to:%s'%target_image)
    print('for neighbors with indexes respectively:')
    print(idx_closest)
    print('\n')
    i=0
    for idx in idx_closest:
        if i>=k:
            print("i was found greater than k. Breaking the loop")
            break
        i+=1
        name=images[idx]
        cut_name=name[-16:]
        print('%d closest with distance %1.2f and name :%s'%(int(i),distances[i-1],cut_name))
        
       

    if display_output:
        query_image = img


        results_image = get_concatenated_images(idx_closest, 200)

        # display the query image
        plt.figure(figsize = (5,5))
        plt.imshow(query_image)

        # display the resulting images
        plt.figure(figsize = (16,12))

        plt.imshow(results_image)
        plt.title("result images")
if use_timing:
    t6 = time.time()    


    
    
    
    

# Timing outputs
###########################################################################
if use_timing:
    Dt1=t1-t0
    Dt2=t2-t1
    Dt3=t3-t2
    Dt4=t4-t3
    Dt5=t5-t4
    Dt6=t6-t5

    print('\n')
    print('\n')
    print('Dt1=%3.3f s:Model parameter loading: '%Dt1)
    
    
    print('%.2f percentage for loading image'%(t11-t1))
    print('%.2f percentage for making image into a numpy array'%(t12-t11))
    print('%.2f percentage for calculating convolutional features'%(t13-t12))
    print('%.2f percentage for making features into a numpy array'%(t11-t1))
    print('Dt2=%3.3f s:1) Image preprocessing and 2)loading of feature calculator 3) feature calculation for one image'%Dt2)
    
    
    print('Dt3=%3.3f s:Choosing the images from the folder:'%Dt3)
    print('Dt4=%3.3f s:Images to convolutional features for %d images'%(Dt4,max_num_images))
    print('Dt5=%3.3f s:PCA '%Dt5)
    print('Dt6=%3.3f s:K nearest neighbohrs'%Dt6)
    print('\n')
    print('\n')


Running search with image:/groups/astro/rami/images/image_000005.png 
 comparing to 20 images 
 which are in folder /groups/astro/rami/images/, 
 using model vgg 
 and distance metric:knn with 5 nearest neighbors. 
 PCA used?:False 
 recalculating dataset?True 


2021-10-03 22:48:45.121063: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-03 22:48:45.122030: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-10-03 22:48:45.147679: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: A100-PCIE-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2021-10-03 22:48:45.149118: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:25:00.0 name: A100-PCIE-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2021-10-03 22:48:45.150526: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 2 with properties: 
pciBusID: 0000:c1:00.0 name: A100-PCIE-40GB com

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

2021-10-03 23:12:58.279584: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


### Just plotting the images on their own cell

In [ ]:
# path='C:\\Users\\Rami\\Desktop\\PetImages\\dogs-vs-cats\\cat_.jpg' 
path='/groups/astro/rami/images/image_000025.png'
img, x = load_image(path=path,model_name=model_name)
x=np.array(x)
feature=img_to_conv_features(model_name=model_name,model=model,x=x)
reduced_feature=np.array(feature)
print("Shape of feature:",np.shape(feature))


if distance_metric=='knn':
    distances,idx_closest=get_neighbors(feature_vector=reduced_feature,feature_vectors=reduced_features,k=k)
    distances=distances.flatten()
    idx_closest=idx_closest.flatten()
    distances=np.array(distances)
    target_image=path[-16:]
    print('neighbour indeces with closest distances with respect to:%s'%target_image)
    print(idx_closest)
    i=0
    for idx in idx_closest:
        if i>=k:
            print("i was found greater than k. Breaking the loop")
            break
        i+=1
        name=images[idx]
        cut_name=name[-16:]
        print('%d closest with distance %1.2f and name :%s'%(int(i),distances[i-1],cut_name))
        

    query_image = img
    

    results_image = get_concatenated_images(idx_closest, 200)
    t6 = time.process_time()    

    # display the query image
    plt.figure(figsize = (5,5))
    plt.imshow(query_image)

    # display the resulting images
    plt.figure(figsize = (16,12))

    plt.imshow(results_image)
    plt.title("result images")
    plt.show()
print(distance_metric)


### Save the  features to pickle_file


In [ ]:
    # !mkdir features
    # print(np.shape(images))
    # print(np.shape(reduced_features))

In [ ]:
# import pickle
# reduced_features=np.array(reduced_features)
# for i in range(len(images)):
#     name=images[i]
#     folder_directory='features/'
#     pickle_name=folder_directory+'feat_vector'+name[-10:-4]+'.pickle'
#     #print(pickle_name)
#     with open(pickle_name, 'wb') as f:
#         save_features=reduced_features[i,:]
#         pickle.dump(save_features, f)

### Read those pickle files

In [ ]:
# import pandas as pd

# object = pd.read_pickle('features/feat_vector000654.pickle')

In [ ]:
# print(np.shape(object))